# Importing packages 

In [1]:
filename = r'20220728_TRR_LSTM_FD'
depth = r'All_Depth'

model = r'LSTM'
trained_on = 'All'
k = 1 #Cluster Number

from datetime import datetime
now = datetime.now()
date_time_string = str(now.year) + str(now.month).zfill(2) + str(now.day).zfill(2) + "_" + str(now.hour).zfill(2) + str(now.minute).zfill(2) + str(now.second).zfill(2)
date_processed = date_time_string



In [2]:
model_folder = r'C:\Users\w10\Desktop\TRR\Final Codes\Freezing Depth\Training\Models'
results_folder = r'C:\Users\w10\Desktop\TRR\Final Codes\Freezing Depth\Training\Results'
pred_results_folder =  r'C:\Users\w10\Desktop\TRR\Final Codes\Freezing Depth\Results'

In [3]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import datetime
import numpy as np
import random as rn
from tensorflow import keras
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Concatenate, SimpleRNN, Masking, Flatten
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import RandomNormal
import os
import xlwt 
from xlwt import Workbook 
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
from tensorflow import keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Activation, BatchNormalization
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard

print('Libraries Loaded')

Libraries Loaded


# Custom functions

In [11]:
def create_dataset(X, time_steps, ts_range):
    '''
    Returns the prepared data based on the lag and look ahead
    
    Parameters:
        X          (float): The independent variables of the data
        y          (float): The dependent variables of the data
        time_steps (int)  : The lag that is being used to lookback
        ts_range   (int)  : The lookahead for the data
    
    Returns:
        Xs (float) : The numpy array of the input variable
        ys (float) : The numpy array of the output variable 
    '''
    Xs = []
    for i in range(len(X) - time_steps - ts_range):
        v = X.iloc[i:(i + time_steps),0:-1].values
        Xs.append(v)
        
    return np.array(Xs)

def splitter(df, features, output, lag,duration):
    '''
    Returns the training and testing data
    
    Parameters:
        df (float): The whole dataframe containing the independent and dependent variables
        output(str): The output variable 
        lag (int): The lag that needs to be applied for the data
        duration (int): The duration that is being considered as output
        ts (float): The percentage of training data
    
    Returns:
        x_train (float): The training data of independent variable 
        x_test (float): The testing data of independent variable
        y_train (float): The training data of the depenedent variable 
        y_test (float): The testing data of the dependent variable 
    '''
    

    pred_size = int(len(df))
    pred = df.iloc[0:pred_size]
    df_pred = pred.copy(deep=True)
    x_pred = create_dataset(df_pred[features],lag,duration)

    return x_pred



class attention(keras.layers.Layer):
    '''
    Attention layer for the neural networks.
    
    if return_sequences=True, it will give 3D vector and if false it will give 2D vector. It is same as LSTMs.

    https://stackoverflow.com/questions/62948332/how-to-add-attention-layer-to-a-bi-lstm/62949137#62949137
    the  following code is being inspired from the above link.
    '''

    def __init__(self, return_sequences=True, **kwargs):
        self.return_sequences = return_sequences
        super(attention, self).__init__()

    def get_config(self):
        cfg = super().get_config()
        return cfg

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")

        super(attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a

        if self.return_sequences:
            return output

        return K.sum(output, axis=1)

# Reading the data 

In [13]:
location = r'NYJ'

src = r'C:\Users\w10\Desktop\TRR\TRR Compiled Data'

prediction_csv_filename = r'NYJ.csv'

predicting_df = pandas.read_csv(os.path.join(src,prediction_csv_filename), engine='python').fillna(0)
predicting_dataset = predicting_df.values


## Specifying the destination path
dest = results_folder
pred_dest = pred_results_folder
features = ['Air_Temperature', 'LPF_1', 'LPF_2', 'LPF_3','RF', 'Wind_Speed', 'Humidity','Solar_Radiation', 'Snow','0mm','100mm','500mm','800mm','1100mm']

prediction = predicting_df[features]

# Data preparation 

In [14]:
lag = 6
n_ahead = 1

all_training_features = ['Air_Temperature', 'LPF_1', 'LPF_2', 'LPF_3', '0mm']

d0mm_training_features = ['Air_Temperature', 'LPF_1', 'LPF_2', 'LPF_3',
                          'RF', 'Wind_Speed', 'Humidity','Solar_Radiation', 'Snow','0mm']

d100mm_training_features = ['Air_Temperature', 'LPF_1', 'LPF_2', 'LPF_3',
                          'RF', 'Wind_Speed', 'Humidity','Solar_Radiation', 'Snow','100mm']

d500mm_training_features = ['Air_Temperature', 'LPF_1', 'LPF_2', 'LPF_3',
                          'RF', 'Wind_Speed', 'Humidity','Solar_Radiation', 'Snow','500mm']

d800mm_training_features = ['Air_Temperature', 'LPF_1', 'LPF_2', 'LPF_3',
                          'RF', 'Wind_Speed', 'Humidity','Solar_Radiation', 'Snow','800mm']

d1100mm_training_features = ['Air_Temperature', 'LPF_1', 'LPF_2', 'LPF_3',
                          'RF', 'Wind_Speed', 'Humidity','Solar_Radiation', 'Snow','1100mm']


x_features = ['Air_Temperature', 'LPF_1', 'LPF_2', 'LPF_3','RF','Wind_Speed', 'Humidity','Solar_Radiation', 'Snow']


d0mm_y_features = ['0mm']
d100mm_y_features = ['100mm']
d500mm_y_features = ['500mm']
d800mm_y_features = ['800mm']
d1100mm_y_features = ['1100mm']

## Creating the matrices for training

In [16]:
## Creating the training and testing data
x_test_0mm= splitter(predicting_df,d0mm_training_features,'0mm',lag,1)
x_test_100mm = splitter(predicting_df,d100mm_training_features,'100mm',lag,1)
x_test_500mm= splitter(predicting_df,d500mm_training_features,'500mm',lag,1)
x_test_800mm= splitter(predicting_df,d800mm_training_features,'800mm',lag,1)
x_test_1100mm= splitter(predicting_df,d1100mm_training_features,'1100mm',lag,1)

# Defining the model 

In [19]:
epochs = 200
batch_size = 64
lr = 0.0001
n_layer = 32
if lag ==1:
    ker_size = 1
else:
    ker_size = 3

In [20]:
def create_model():
    lstm = keras.Sequential()
    lstm.add(keras.layers.LSTM(n_layer, return_sequences=True, input_shape=(x_test_0mm.shape[1], x_test_0mm.shape[2])))
    lstm.add(keras.layers.LSTM(n_layer, return_sequences=True))
    lstm.add(keras.layers.Flatten())
    lstm.add(keras.layers.Dense(32))
    lstm.add(keras.layers.Dense(1))
    return lstm

from keras.models import load_model

d0mm_model_path = r'C:\Users\w10\Desktop\TRR\Final Codes\Freezing Depth\Training\Models\20220729_034243_ChunCheon_20220728_TRR_LSTM_FD_0mm.hdf5'
d100mm_model_path = r'C:\Users\w10\Desktop\TRR\Final Codes\Freezing Depth\Training\Models\20220729_034312_ChunCheon_20220728_TRR_LSTM_FD_100mm.hdf5'
d500mm_model_path = r'C:\Users\w10\Desktop\TRR\Final Codes\Freezing Depth\Training\Models\20220729_034402_ChunCheon_20220728_TRR_LSTM_FD_500mm.hdf5'
d800mm_model_path = r'C:\Users\w10\Desktop\TRR\Final Codes\Freezing Depth\Training\Models\20220729_034428_ChunCheon_20220728_TRR_LSTM_FD_800mm.hdf5'
d1100mm_model_path = r'C:\Users\w10\Desktop\TRR\Final Codes\Freezing Depth\Training\Models\20220729_034449_ChunCheon_20220728_TRR_LSTM_FD_1100mm.hdf5'
    
    
# model = load_model(filepath_attention)
def load_trained_model(path):
    model = create_model()
    model.load_weights(path)
    return model

d0mm_final_model = load_trained_model(d0mm_model_path)
d100mm_final_model = load_trained_model(d100mm_model_path)
d500mm_final_model = load_trained_model(d500mm_model_path)
d800mm_final_model = load_trained_model(d800mm_model_path)
d1100mm_final_model = load_trained_model(d1100mm_model_path)


d0mm_preds = d0mm_final_model.predict(x_test_0mm)
d100mm_preds = d100mm_final_model.predict(x_test_100mm)
d500mm_preds = d500mm_final_model.predict(x_test_500mm)
d800mm_preds = d800mm_final_model.predict(x_test_800mm)
d1100mm_preds = d1100mm_final_model.predict(x_test_1100mm)

print('Prediction Complete')

486/486 [==============================] - 2s 2ms/step
Prediction Complete


In [21]:
y_actual_0mm = predicting_df['0mm']
y_actual_100mm = predicting_df['100mm']
y_actual_500mm = predicting_df['500mm']
y_actual_800mm = predicting_df['800mm']
y_actual_1100mm = predicting_df['1100mm']

days = pd.DataFrame(predicting_df['Date'].values[-len(d0mm_preds):])

In [22]:
y_pred_0mm = pd.DataFrame([x[0] for x in d0mm_preds])

y_pred_100mm = pd.DataFrame([x[0] for x in d100mm_preds])

y_pred_500mm = pd.DataFrame([x[0] for x in d500mm_preds])

y_pred_800mm = pd.DataFrame([x[0] for x in d800mm_preds])

y_pred_1100mm = pd.DataFrame([x[0] for x in d1100mm_preds])

In [23]:
test_frame = pd.concat([days, y_actual_0mm, y_pred_0mm,y_actual_100mm, y_pred_100mm,y_actual_500mm, y_pred_500mm,
                       y_actual_800mm, y_pred_800mm,y_actual_1100mm, y_pred_1100mm,], axis = 1)
test_frame.columns = ['Date', 'Actual_0mm', 'Predicted_0mm','Actual_100mm', 'Predicted_100mm','Actual_500mm', 'Predicted_500mm',
                     'Actual_800mm', 'Predicted_800mm','Actual_1100mm', 'Predicted_1100mm']


In [24]:
min_temp_difference_0mm = test_frame.Actual_0mm.dropna().min()- test_frame.Predicted_0mm.dropna().min()
min_temp_difference_100mm = test_frame.Actual_100mm.dropna().min()- test_frame.Predicted_100mm.dropna().min()
min_temp_difference_500mm = test_frame.Actual_500mm.dropna().min()- test_frame.Predicted_500mm.dropna().min()
min_temp_difference_800mm = test_frame.Actual_800mm.dropna().min()- test_frame.Predicted_800mm.dropna().min()
min_temp_difference_1100mm = test_frame.Actual_1100mm.dropna().min()- test_frame.Predicted_1100mm.dropna().min()

print(min_temp_difference_0mm)
print(min_temp_difference_100mm)
print(min_temp_difference_500mm)
print(min_temp_difference_800mm)
print(min_temp_difference_1100mm)

0.9250421524047852
-2.174933910369873
-0.7172691226005554
0.2624317407608032
0.463062047958374


In [25]:
print(f'0mm Predicted Minimum Temperature: {test_frame.Predicted_0mm.dropna().min()}')
print(f'0mm Actual Minimum Temperature: {test_frame.Actual_0mm.dropna().min()}')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

print(f'100mm Predicted Minimum Temperature: {test_frame.Predicted_100mm.dropna().min()}')
print(f'100mm Actual Minimum Temperature: {test_frame.Actual_100mm.dropna().min()}')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

print(f'500mm Predicted Minimum Temperature: {test_frame.Predicted_500mm.dropna().min()}')
print(f'500mm Actual Minimum Temperature: {test_frame.Actual_500mm.dropna().min()}')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

print(f'800mm Predicted Minimum Temperature: {test_frame.Predicted_800mm.dropna().min()}')
print(f'800mm Actual Minimum Temperature: {test_frame.Actual_800mm.dropna().min()}')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

print(f'1100mm Predicted Minimum Temperature: {test_frame.Predicted_1100mm.dropna().min()}')
print(f'1100mm Actual Minimum Temperature: {test_frame.Actual_1100mm.dropna().min()}')
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

0mm Predicted Minimum Temperature: -11.425042152404785
0mm Actual Minimum Temperature: -10.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
100mm Predicted Minimum Temperature: -5.825066089630127
100mm Actual Minimum Temperature: -8.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
500mm Predicted Minimum Temperature: -0.2827308773994446
500mm Actual Minimum Temperature: -1.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
800mm Predicted Minimum Temperature: 1.2375682592391968
800mm Actual Minimum Temperature: 1.5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1100mm Predicted Minimum Temperature: 2.536937952041626
1100mm Actual Minimum Temperature: 3.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
